# How to ... Grover ?

Le circuit quantique de Grover est découpé en plusieurs morceaux :
- La QRAM pour le mapping du dataset
- L'algo pour les transformations générales
- L'oracle qui va vérifier que le résultat de l'algo correspond à ce qui est attendu et va appliquer une transformation sur les data qui passent l'algo
- On reverse l'algo et la QRAM pour annuler les transformations
- Le diffuseur qui va venir chercher quel data a été transformer par l'oracle
- La mesure de la qram

![Full Grover](resources/full_grover.png)

Quand on commence notre circuit il faut initialiser certains qubits :
- La Qram : Afin de générer tous les état possible sur notre Qram il est nécessaire de mettre les qubits de celle-ci en superposition via une _porte Hadamard_ (_porte H_).
- L'Oracle : C'est lui qui va dire quel data est bonne ou pas, pour lui permettre de faire cela on a besoin de lui appliquer une transformation de $-\pi/2$ sur sa phase, pour ce faire on applique simplement une _porte X_ et une _porte H_ (dans cet ordre)

## Qram

Le role de la QRAM est de mapper les data du dataset dans le circuit. Pour faire cela, on donne une adresse binaire à chaque data du dataset et en fonction de cette adresse on applique différentes transformation dur les qubits de la partie algo.

En plus clair :
Chaque data du dataset : `[95, 100, 90, 0]` aura une adresse dans la QRAM
- 95 correspondra à l'adresse 00
- 100 à 01
- 90 à 10
- 0 à 11

Une adresse correspond à la valeur des QRAM à un instant T.

---

Pour savoir à l'instant T si les qubits ont cette valeur on pose des _portes Not_ (aussi appelé _porte X_), ce sont des portes qui font passer 1 qubit de 0 à 1 et inversement.

Et donc pour vérifier que l'adresse de nos 2 qubits correspondent à 00 on va mettre une _porte Not_ sur chacune d'elle. De cette manière si nos 2 qubits on la valeur attendu alors ils seront tous 2 à 1. En sachant cela on peut utiliser des _portes de control_ qui ne fonctionnent que si les qubits controlent sont à 1.
Pour finir on réapplique nos _portes Not_ pour annuler cette action. Ensuite on fait pareil pour les autres combinaisons.

Et dans l'ordre celà donne ceux ci :
![MCT Qram](resources/qram_simple.png)

Sur cette figure les portes vertes sont les _portes Not_, permettant d'etre sur de vérifier les valeurs et donc l'adresse de nos qubits et les portes violetts sont des _portes multi tofoli_ (aussi appelé _porte CCX_) elles ne s'appliquent que si les qubits de la QRAM sont à 1.

Ici les transformations appliqués par les _portes CCX_ sont toutes identiques et font passer le qubit cible à 1 si les controlent sont aussi à 1. Dans la pratique on ne met jamais la même transformation pour chaque data mais j'ai pas trouvé d'autre portes qui présentent bien les controles ^^'

---

Dans le programme [Grover get minimal value](grover_min_data.ipynb), j'ai utilisé une _porte mcrx_ c'est aussi une _porte multi tofoli_ mais qui applique une transformation de $\pi$ sur un angle $\theta$ donné en fonction de la valeur de la data mais elle ne possède pas de vision schematic en un seul bloc. Mais dit toi que chacun des groupement de portes ci-dessous (entre chacun _porte Not_) correspond à la même que sur la fig précente mais avec une transformation différente en fonction de la valeur de la data et ne s'applique que si les qubits de la qram sont à 1 :
![MCRX Qram](resources/qram_complex.png)

Pour trouver la valeur la plus faible du dataset, je suis partie du principe que chaqu'une des data allait appliquer une erreur en pourcent sur une rotation de $\pi$. Faire une rotation de $\pi$ sur l'angle $\theta$ permet à un qubit de passer de 0 à 1 et inversement.

Du coup voilà ma formule :  $\arccos(1-((100-d)*2/100))$
- $d$ correspond à la data.
    - si $d = 0$ alors aucune erreur de rotation ne sera appliqué
    - si $d = 90$ alors une rotation de $10\%$ sera appliqué
    - ...
- $\arccos$ de -1 donne une rotation de $\pi$
- $\arccos$ de 1 ne fait rien du tout
- $\arccos$ de 0 donne une rotation de $\pi/2$

## Algo

Le role du bloc algo est d'appliquer la même transformation à chacune des data. C'est ce que j'appel des transformations génériques. Cela peut etre un compteur d'état fait en liant des qubits entre eux, ou autre. Dans mon jeu Pokemon c'est à ce niveau là que je map les faiblesses et résistances.

On peut considérer que la Qram fait aussi partie de l'algo car les transformations sont faites sur le qubit de celui-ci.

---

Dans [Grover get minimal value](grover_min_data.ipynb), je n'y applique rien car je n'avais rien à y appliquer ^^' (pour l'instant)

## Oracle

L'oracle, c'est lui qui va vérifier qu'une data est valide ou non. En gros il va venir vérifier l'état des qubits de l'algo et venir dire et si ceux-ci correspondent alors il va venir appliquer une tranformation de $-\pi/4$ (4 correspond à la taille du dataset) sur l'amplitude à l'adresse de la Qram en question (il vient remonter toute la chaine de qubits pour venir appliquer son état aux qubits de la qram).

![Step oracle](resources/oracle_step.jpeg)

Ici en violet on peut voir l'application de l'oracle sur la data.

---
Dans [Grover get minimal value](grover_min_data.ipynb), je lui fait juste vérifier si le qubit d'algo est bien à 1, si oui alors il va appliquer son état à la qram.

## Reverse

L'important de Grover c'est de ne garder QUE la transformation fait par l'oracle et ne jamais garder les autres, du coup il faut reverse toutes les autres transformations dans l'ordre inverse qu'on les a appliqué.

Ca peut paraitre compliquer pour certaines portes qui ne sont pas réversibles mais pour ca il y a cette fonction : `.inverse()` ;)

## Diffuseur

Le diffuseur a pour role de trouver sur quel adresse qram l'oracle a fait sa transformation. Il va donc faire la chasse aux phases négatives et reverse cette phase plus lui donner un boost de 2 fois son amplitude négative.

![Step diffuseur](resources/diffuseur_step.jpeg)

Ici en violet on peut voir l'action du diffuseur.


## Mesures & results

Enfin pour finir on mesure simplement notre Qram et on execute notre circuit un bon millier de fois.

L'adresse qui sera la plus dans les réponses sera le résulat de notre circuit. Si plusieurs réponses sont attendu alors celle-si seront plus présentes que les autres.

Il faut noter que plus notre dataset est grand ou plus de bonnes réponses sont possibles et plus il sera nécessaire de répeter les block Algo et Qram de notre circuit. Un nombre optimimal d'iteration existe et à ne pas dépaser sous peine de voir les bonnes réponses masquer par les mauvaises.

---

Dans [Grover get minimal value](grover_min_data.ipynb), j'applique une erreur de rotation pour chacune de mes data, ce qui veut dire que les grosses valeurs on toujours une chance d'etre bonne (hors 100 qui correspond à 100$\%$ d'erreur). De ce fait il faudra un nombre suffisant d'iteration + un dataset adapté afin de que mon circuit fonctionne comme attendu. Une amélioration serait d'augmenter le nombre qubit algo afin d'augmenter artificiellement les taux d'erreur appliqué.

Enfin un gros probleme d'algo est aussi présent, plus on aura d'iterations et plus les grosses valeurs auront une chance de passer il vaut mieux donc n'en avoir qu'une pour cet algo.
Je viens de modifier le programme en conséquence et cela permet plus d'avoir des valeurs prochent.

Je pense que revoir tout l'algo pourrait aussi servir, ici je suis limité à des valeurs entre 0 et 100. Faire un décrementeur de centaines, puis de dizaines puis d'unités pourraient etre une idée...